In [1]:
#The 3 following cells aim at simuling the behavior of NN models with a simple model for MNIST

EPOCH_SIZE = 512*32
TEST_SIZE = 256*32

#This is a function that can be used by several NN model
def train(model, optimizer ,func ,train_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.train()
    #for (data, target) in train_loader:
    for batch_idx, (data, target) in enumerate(train_loader):
        # We set this just for the example to run quickly.
        if batch_idx * len(data) > EPOCH_SIZE:
           # print("hehe")
            return
        # We set this just for the example to run quickly.
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = func(output, target)
        loss.backward()
        optimizer.step()
        
#This is a function that can be used by several NN model (it only does accuracy ATM)
def test(model, func, data_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(data_loader):
            # We set this just for the example to run quickly.
            data, target = data.to(device), target.to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)

            total += target.size(0)
            correct += (predicted == target).sum().item()


                
    return correct / total

In [2]:
# A random mnist from the internet to get a correct model to reason about

class train_mnist():
    def __init__(self,config):
        
        self.config = {
         "weight_decay": .017
       , "sigmoid_func": 1
      ,  "hidden_dim":43
      ,  "n_layer":2
      ,  "droupout_prob":0.28     
    }
        for key, value in config.items():
            self.config[key] = value
        config = self.config
        
        self.i = 0
        
        mnist_transforms = transforms.Compose(
            [transforms.ToTensor(),
             transforms.Normalize((0.1307, ), (0.3081, ))])

        self.train_loader = DataLoader(
            datasets.MNIST("~/data", train=True, download=True, transform=mnist_transforms),
            batch_size=64,
            shuffle=True)
        self.test_loader = DataLoader(
            datasets.MNIST("~/data", train=False, transform=mnist_transforms),
            batch_size=64,
            shuffle=True)

        sigmoid_func_uniq = nn.Tanh()


        self.model = LeNet(192,int(round(config.get("hidden_dim",64))),10,
                    int( round(config.get("n_layer",1))),
                    config.get("droupout_prob",0.1) ,sigmoid_func_uniq)
        
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=config.get("lr", 0.01), 
                                     betas=((config.get("b1", 0.999),config.get("b2", 0.9999))),
                                    eps=config.get("eps", 1e-08), 
                                     weight_decay=config.get("weight_decay", 0), 
                                     amsgrad=True)

    
    def adapt(self, config):
        for key, value in config.items():
            self.config[key] = value
        config = self.config
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=config.get("lr", 0.01), 
                                     betas=((config.get("b1", 0.999),config.get("b2", 0.9999))),
                                    eps=config.get("eps", 1e-08), 
                                     weight_decay=config.get("weight_decay", 0), 
                                     amsgrad=True)
        return copy.deepcopy(self)
    
    
# All NN models should have a function train1 and test1 that calls the common train and test defined above.
# train1 and test1 is then used in the scheduler
    def train1(self):
        print("iteration: " + str(self.i) )
        self.i+=1
        train(self.model, self.optimizer, F.nll_loss, self.train_loader)

    def test1(self):
        return test(self.model, F.nll_loss, self.test_loader)

# This should be a hyperspace instead of constants.

# __INCEPTION_SCORE_begin__
class LeNet(nn.Module):
    """
    LeNet for MNist classification, used for inception_score
    """

    def __init__(self,input_dim, hidden_dim, output_dim, n_layers,
                 drop_prob, sigmoid ):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)



# Convolution Neural network using Pytorch 
class ConvNet(nn.Module):
    def __init__(self,input_dim, hidden_dim, output_dim, n_layers,
                 drop_prob, sigmoid ):
        super(ConvNet, self).__init__()
        
        self.sigmoid = sigmoid
        self.i_d = input_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.conv1 = nn.Conv2d(1, 3, kernel_size=3)

        self.fc = nn.Linear(input_dim, output_dim)
        self.first= nn.Linear(input_dim, hidden_dim)
        self.hidden = [nn.Linear(hidden_dim,hidden_dim) for _ in range(self.n_layers)]
        self.drop_out = nn.Dropout(drop_prob)

        self.last = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.sigmoid(F.max_pool2d(self.conv1(x), 3))
        x = x.view(-1, self.i_d)
        x=self.first(x)
        x=self.drop_out(x)
        for i in range(self.n_layers):
            x=self.hidden[i](x)
            x=self.drop_out(x)
        x = self.last(x)
        return F.log_softmax(x, dim=1)


NameError: name 'nn' is not defined

In [ ]:
config_linear_reg = {
    "lr": 0.031,
    "l2_regularization": .01,
    "minibatch_size": 512,
    "hidden_dims": [150, 100, 75],
    "droupout_prob": 0.28,
    "weight_decay": .017,
    "batch_size": 64
}

class LinearReg(nn.Module):
    def __init__(self, config):
        super(LinearReg, self).__init__()

        hidden_dims = config.get("hidden_dims", [150, 100, 75])
        self.linears = nn.ModuleList([nn.Linear(28*28, hidden_dims[0], bias=True), nn.ReLU()])
        for i in range(1, len(hidden_dims) ):
            self.linears.append(nn.Linear(hidden_dims[i-1], hidden_dims[i], bias=True))
            self.linears.append(nn.ReLU())
        self.model = Net(self.linears) 

        mnist_transforms = transforms.Compose([transforms.ToTensor(), \
                                               transforms.Normalize((0.1307,), (0.3081,))])

        self.train_loader = DataLoader(datasets.MNIST("~/data", train=True, download=True, transform=mnist_transforms),\
                                       batch_size=config.get("batch_size", 64), shuffle=True)

        self.test_loader = DataLoader(datasets.MNIST("~/data", train=False, transform=mnist_transforms),\
                                      batch_size=config.get("batch_size", 64), shuffle=True)
        
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=config.get("lr", 0.01), 
                                     betas=((config.get("b1", 0.999),config.get("b2", 0.9999))),
                                    eps=config.get("eps", 1e-08), 
                                     weight_decay=config.get("weight_decay", 0), 
                                     amsgrad=True)
    def adapt(self, config):
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=config.get("lr", 0.01), \
                                          betas=((config.get("b1", 0.999),config.get("b2", 0.9999))),\
                                          eps=config.get("eps", 1e-08), weight_decay=config.get("weight_decay", 0), \
                                          amsgrad=True)
    def forward(self, x):
        for i, layer in enumerate(self.linears):
            x = l(x)
        return x     
    
    def train1(self):
        train(self.model, self.optimizer, F.nll_loss, self.train_loader)

    def test1(self):
        return test(self.model, F.nll_loss, self.test_loader)
    
    
class Net(nn.Module):
    def __init__(self,linears):
        super(Net, self).__init__()
        self.linears = linears

    def forward(self, x):
        for i, layer in enumerate(self.linears):
            x = layer(x)
        return x 


In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch
import torch.nn.functional as F
import math
import numpy as np
import random
import math 


In [ ]:
config = {
    "lr": 0.031,
    "l2_regularization": .01,
    "minibatch_size": 512,
    "hidden_dims": [150, 100, 75],
    "droupout_prob": 0.28,
    "weight_decay": .017,
    "batch_size": 64
}

a = LinearReg(config)
a.train1()
print(a.test1())


In [ ]:
# A random mnist from the internet to get a correct model to reason about

class toy():
    def __init__(self,config):
        self.hyperparameter = config.get("lr",0)
    
    def adapt(self,config):
        self.hyperparameter = config.get("lr",0)
        return self
    
    def train1(self):
        self.hyperparameter = self.hyperparameter * 2 
    def test1(self):
        return self.hyperparameter

# This should be a hyperspace instead of constants.
config= {
     "lr": 0.031
}
a = train_mnist(config)
a.train1()
print(a.test1())

class Parent():
    def __init__(self,hyperspace,configuration, model, loss):
        
        self.hyperspace = hyperspace
        self.configuration_list = np.array(configuration) 
        self.loss_list = [np.array(loss)] 
        self.model = model
    
    def update(self,configuration,loss, model):
        self.configuration_list = np.append(self.configuration_list,configuration) 
        self.loss_list = np.append(self.loss_list,loss)
        self.model = model
    
    def get_hyperspace(self):
        return self.hyperspace

    def get_model(self):
        return self.model
    
    def get_loss(self):
        return self.loss_list

In [ ]:
a.train1()
print(a.test1())


In [ ]:
a.train1()
print(a.test1())


In [3]:
import copy


class Scheduler():
    def __init__(self, model, num_iteration, num_config,
                 oracle):
        #Oracle manages the Bayesian optimization
        self.oracle = oracle
        self.iteration = num_iteration
        self.num_config = num_config 
        self.sqrt_config = math.ceil(num_config/10) #math.floor(math.sqrt(num_config))
        
        #self.h is for the m "h" used at every loop, h is a configuration from the search space
        self.h = np.repeat({},num_config) 
        
        #self.out is for storing the result of the algorithm, ie all "h" from all iterations
        #from all sqrt(m) best models per iterations.
        self.out = np.zeros((num_iteration,self.sqrt_config))
        
        #self.hyperspaces is for storing the sqrt(m) hyperspaces used by the algorithm
        self.hyperspaces = np.zeros(self.sqrt_config)
        

        
        #self.model is the m model that will explore new hyperspace points at every iterations
        self.models = np.repeat(model,num_config)
        
        #self.parents is the sqrt(m) best model from last iteration
        self.parents = np.repeat(model,self.sqrt_config)

        #self.losses remembers the performances of all m models at one iteration to decide which ones are the sqrt(m) best from self.models.
        self.losses = np.zeros(num_config)
        
        self.k = [0] # c'est pour avoir un pointeur sur k, c'est pas paralélisable pour le moment du coup.
    
    def initialisation(self):
        num_config = self.num_config
        extended_Hyperspace = Trials()
        #for k in range(num_config):
        fmin_objective = partial(test_function, models=self.models,h=self.h,losses=self.losses,parent_model=self.models, k_f = self.k,iteration = 0)
        self.oracle.compute_Batch(extended_Hyperspace ,num_config , 0 ,fmin_objective)
            
        indexes = np.argsort(self.losses)     
        self.out[0] = (self.losses[indexes])[0:self.sqrt_config]
        self.hyperspaces = np.repeat(extended_Hyperspace,self.sqrt_config)    
        self.parents = np.array([Parent(copy.deepcopy(extended_Hyperspace),(self.h[indexes])[i], (self.models[indexes])[i],(self.losses[indexes])[i])  
                                 for i in range(self.sqrt_config) ])         
        
    def loop(self):
        sqrt_config = self.sqrt_config
        iteration = self.iteration
        for i in range(1,iteration):
            for j in range(sqrt_config):
                k = j*sqrt_config
                parent = self.parents[j]
                extended_Hyperspace = parent.get_hyperspace()
                print("\n loss of parent " + str(parent.get_loss()[-1]) )
                print("\n loss " + str(parent.get_loss()))

                
                for l in range(int(self.num_config/sqrt_config)):
                    fmin_objective = partial(test_function, models=self.models,h=self.h,losses=self.losses,parent_model=parent.get_model(), k_f = k,iteration = i)
                    self.oracle.compute_Once(extended_Hyperspace ,i ,fmin_objective)
                    k = k+1
                
            indexes = np.argsort(self.losses)     
            #parent_idx = np.floor(indexes/sqrt_config)[:sqrt_config]
            parent_idx = indexes[:sqrt_config]
            print(self.losses)
            print(indexes)
            #print(parent_idx)
 
            temp = np.empty(self.sqrt_config, dtype=Parent)
            for a,x in enumerate(parent_idx):
                x = int(x)
                temp[a] = copy.deepcopy(self.parents[math.floor(x/self.num_config*sqrt_config)])
                temp[a].update(self.h[x], self.losses[x], self.models[x])
            self.parents = temp
  


    
    
CONFIGURATION = 10
ITERATIONS = 4
config= {
     "lr": hp.uniform("lr",0,1)
}
model = train_mnist
oracle = Oracle(config)
scheduler = Scheduler(
    model,
    ITERATIONS,
    CONFIGURATION,
    oracle) 

scheduler.initialisation()     
scheduler.loop()     


NameError: name 'hp' is not defined

In [ ]:
from hyperopt import hp, fmin, tpe, Trials

# Oracle (Paul) TODO
class Oracle():
    def __init__(self, searchspace ):
        #self.hyperspace is the original (input) searchspace
        self.searchspace = searchspace

    def compute_Once(self,trials, iteration,function): #add space
        space = copy.deepcopy(self.searchspace)
        curr_eval = getattr(trials,'_ids')
        if curr_eval == set():
            curr_eval = 0
        else:
            curr_eval = max(curr_eval) +1
        space["itération"] =  hp.quniform("itération",-.5+iteration,.5+iteration, 1) 
        fmin(function, space, algo=partial(tpe.suggest, n_startup_jobs=1), max_evals=curr_eval
+1, trials=trials)
        
        
    def compute_Batch(self,trials, nb_eval, iteration,function): #add space
        space = copy.deepcopy(self.searchspace)
        curr_eval = getattr(trials,'_ids')
        if curr_eval == set():
            curr_eval = 0
        else:
            curr_eval = max(curr_eval) +1
            
        space["itération"] =  hp.quniform("itération",-.5+iteration,.5+iteration, 1) 
        fmin(function, space, algo=partial(tpe.suggest, n_startup_jobs=1), max_evals=curr_eval
+nb_eval, trials=trials)
    


In [ ]:
def test_function(x,models,h,losses, parent_model,k_f,iteration):
    if iteration == 0:
        k = k_f[0]
        models[k] = parent_model[k](x)
        k_f[0] += 1

    else:      
        k = k_f
        models[k] = parent_model.adapt(x)
    print("\n lr, " + str(x["lr"]))
    h[k] = x
    models[k].train1()
    loss = models[k].test1()
    losses[k] = -loss
    print("accuracy, " + str(loss))
    return -loss

In [ ]:
#Normal Bayes Opt

def function(x):
    print(x)
    model = train_mnist(x)
    for _ in range(4):
            model.train1()
    loss = model.test1()
    return -loss

In [ ]:
class BayesOpt():
    def __init__(self, searchspace ):
        self.searchspace = searchspace

    def compute_Once(self,function): 
        fmin(function, self.searchspace, algo=partial(tpe.suggest, n_startup_jobs=1), max_evals=9, trials=Trials())
    
config= {
     "lr": hp.uniform("lr",0,1)
}
oracle = BayesOpt(config)

In [ ]:
oracle.compute_Once(function)

In [13]:
# IN THIS MODULE: IMPORTS, CNN, TRAIN, TEST, MNIS_FUNCTION, SPACE

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
from ray import tune
from ray.tune.schedulers import ASHAScheduler
from hyperopt import hp
from ray.tune.suggest.hyperopt import HyperOptSearch
from ray.tune.suggest.skopt import SkOptSearch
from ray import tune
from ray.tune.suggest.bayesopt import BayesOptSearch
import time
import ray
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest.ax import AxSearch
import argparse
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest.nevergrad import NevergradSearch
import nevergrad as ng
import json
import os
from ray.tune import Trainable
from ray.tune.schedulers.hb_bohb import HyperBandForBOHB
from ray.tune.suggest.bohb import TuneBOHB
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest.dragonfly import DragonflySearch
from ray.tune.suggest.zoopt import ZOOptSearch
from ray.tune.schedulers import AsyncHyperBandScheduler
from zoopt import ValueType
import torch
# IN THIS MODULE: IMPORTS, CNN, TRAIN, TEST, MNIS_FUNCTION, SPACE

import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
from ray import tune
from ray.tune.schedulers import ASHAScheduler 
from ray.tune.schedulers import MedianStoppingRule

from hyperopt import hp
from ray.tune.suggest.hyperopt import HyperOptSearch
from ray.tune.suggest.skopt import SkOptSearch
from ray import tune
from ray.tune.suggest.bayesopt import BayesOptSearch
import time
import ray
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest.ax import AxSearch
import argparse
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest.nevergrad import NevergradSearch
import nevergrad as ng
import json
import os
from ray.tune import Trainable
from ray.tune.schedulers.hb_bohb import HyperBandForBOHB
from ray.tune.suggest.bohb import TuneBOHB
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.schedulers import AsyncHyperBandScheduler
#from ray.tune.suggest.dragonfly import DragonflySearch
from ray.tune.suggest.zoopt import ZOOptSearch
from ray.tune.schedulers import AsyncHyperBandScheduler
from zoopt import ValueType
import torch
import adabelief_pytorch
global_checkpoint_period=np.inf

def GAN_MNIST(SA):
    import ray

    import os
    import torch
    import torch.nn as nn
    import torch.nn.parallel
    import torch.utils.data
    import torchvision.datasets as dset
    import torchvision.transforms as transforms
    import torchvision.utils as vutils
    import numpy as np

    import ray
    from ray import tune
    from ray.tune.trial import ExportFormat
    from ray.tune.schedulers import PopulationBasedTraining

    import argparse
    import os
    from filelock import FileLock
    import random
    import torch
    import torch.nn as nn
    import torch.nn.parallel
    import torch.optim as optim
    import torch.utils.data
    import numpy as np
    from ray.tune.suggest.bayesopt import BayesOptSearch
    from ray.tune.suggest.ax import AxSearch



    from torch.autograd import Variable
    from torch.nn import functional as F
    from scipy.stats import entropy

    import matplotlib.pyplot as plt
    import matplotlib.animation as animation

    # Training parameters
    dataroot = ray.utils.get_user_temp_dir() + os.sep
    workers = 2
    batch_size = 64
    image_size = 32

    # Number of channels in the training images. For color images this is 3
    nc = 1

    # Size of z latent vector (i.e. size of generator input)
    nz = 100

    # Size of feature maps in generator
    ngf = 32

    # Size of feature maps in discriminator
    ndf = 32

    # Beta1 hyperparam for Adam optimizers
    beta1 = 0.5

    # iterations of actual training in each Trainable _train
    train_iterations_per_step = 5

    MODEL_PATH = os.path.expanduser("~/.ray/models/mnist_cnn.pt")


    def get_data_loader():
        dataset = dset.MNIST(
            root=dataroot,
            download=True,
            transform=transforms.Compose([
                transforms.Resize(image_size),
                transforms.ToTensor(),
                transforms.Normalize((0.5, ), (0.5, )),
            ]))

        # Create the dataloader
        dataloader = torch.utils.data.DataLoader(
            dataset, batch_size=batch_size, shuffle=True, num_workers=workers)

        return dataloader


    # __GANmodel_begin__
    # custom weights initialization called on netG and netD
    def weights_init(m):
        classname = m.__class__.__name__
        if classname.find("Conv") != -1:
            nn.init.normal_(m.weight.data, 0.0, 0.02)
        elif classname.find("BatchNorm") != -1:
            nn.init.normal_(m.weight.data, 1.0, 0.02)
            nn.init.constant_(m.bias.data, 0)


    # Generator Code
    class Generator(nn.Module):
        def __init__(self):
            super(Generator, self).__init__()
            self.main = nn.Sequential(
                # input is Z, going into a convolution
                nn.ConvTranspose2d(nz, ngf * 4, 4, 1, 0, bias=False),
                nn.BatchNorm2d(ngf * 4),
                nn.ReLU(True),
                nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
                nn.BatchNorm2d(ngf * 2),
                nn.ReLU(True),
                nn.ConvTranspose2d(ngf * 2, ngf, 4, 2, 1, bias=False),
                nn.BatchNorm2d(ngf),
                nn.ReLU(True),
                nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False),
                nn.Tanh())

        def forward(self, input):
            return self.main(input)


    class Discriminator(nn.Module):
        def __init__(self):
            super(Discriminator, self).__init__()
            self.main = nn.Sequential(
                nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
                nn.LeakyReLU(0.2, inplace=True),
                nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
                nn.BatchNorm2d(ndf * 2), nn.LeakyReLU(0.2, inplace=True),
                nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
                nn.BatchNorm2d(ndf * 4), nn.LeakyReLU(0.2, inplace=True),
                nn.Conv2d(ndf * 4, 1, 4, 1, 0, bias=False), nn.Sigmoid())

        def forward(self, input):
            return self.main(input)


    # __GANmodel_end__


    # __INCEPTION_SCORE_begin__
    class Net(nn.Module):
        """
        LeNet for MNist classification, used for inception_score
        """

        def __init__(self):
            super(Net, self).__init__()
            self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
            self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
            self.conv2_drop = nn.Dropout2d()
            self.fc1 = nn.Linear(320, 50)
            self.fc2 = nn.Linear(50, 10)

        def forward(self, x):
            x = F.relu(F.max_pool2d(self.conv1(x), 2))
            x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
            x = x.view(-1, 320)
            x = F.relu(self.fc1(x))
            x = F.dropout(x, training=self.training)
            x = self.fc2(x)
            return F.log_softmax(x, dim=1)


    def inception_score(imgs, mnist_model_ref, batch_size=32, splits=1):
        N = len(imgs)
        dtype = torch.FloatTensor
        dataloader = torch.utils.data.DataLoader(imgs, batch_size=batch_size)
        cm = ray.get(mnist_model_ref)  # Get the mnist model from Ray object store.
        up = nn.Upsample(size=(28, 28), mode="bilinear").type(dtype)

        def get_pred(x):
            x = up(x)
            x = cm(x)
            return F.softmax(x).data.cpu().numpy()

        preds = np.zeros((N, 10))
        for i, batch in enumerate(dataloader, 0):
            batch = batch.type(dtype)
            batchv = Variable(batch)
            batch_size_i = batch.size()[0]
            preds[i * batch_size:i * batch_size + batch_size_i] = get_pred(batchv)

        # Now compute the mean kl-div
        split_scores = []
        for k in range(splits):
            part = preds[k * (N // splits):(k + 1) * (N // splits), :]
            py = np.mean(part, axis=0)
            scores = []
            for i in range(part.shape[0]):
                pyx = part[i, :]
                scores.append(entropy(pyx, py))
            split_scores.append(np.exp(np.mean(scores)))

        return np.mean(split_scores), np.std(split_scores)


    # __INCEPTION_SCORE_end__


    def train(netD, netG, optimG, optimD, criterion, dataloader, iteration, device,
              mnist_model_ref):
        real_label = 1
        fake_label = 0

        for i, data in enumerate(dataloader, 0):
            if i >= train_iterations_per_step:
                break

            netD.zero_grad()
            real_cpu = data[0].to(device)
            b_size = real_cpu.size(0)
            label = torch.full(
                (b_size, ), real_label, dtype=torch.float, device=device)
            output = netD(real_cpu).view(-1)
            errD_real = criterion(output, label)
            errD_real.backward()
            D_x = output.mean().item()

            noise = torch.randn(b_size, nz, 1, 1, device=device)
            fake = netG(noise)
            label.fill_(fake_label)
            output = netD(fake.detach()).view(-1)
            errD_fake = criterion(output, label)
            errD_fake.backward()
            D_G_z1 = output.mean().item()
            errD = errD_real + errD_fake
            optimD.step()

            netG.zero_grad()
            label.fill_(real_label)
            output = netD(fake).view(-1)
            errG = criterion(output, label)
            errG.backward()
            D_G_z2 = output.mean().item()
            optimG.step()

            is_score, is_std = inception_score(fake, mnist_model_ref)

            # Output training stats
           # if iteration % 10 == 0:
            #    print("[%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z))"
             #         ": %.4f / %.4f \tInception score: %.4f" %
              #        (iteration, len(dataloader), errD.item(), errG.item(), D_x,
               #        D_G_z1, D_G_z2, is_score))

        return errG.item(), errD.item(), is_score


    def plot_images(dataloader):
        # Plot some training images
        if 1==0:
            real_batch = next(iter(dataloader))
            plt.figure(figsize=(8, 8))
            plt.axis("off")
            plt.title("Original Images")
            plt.imshow(
                np.transpose(
                    vutils.make_grid(real_batch[0][:64], padding=2,
                                     normalize=True).cpu(), (1, 2, 0)))

            plt.show()


    def demo_gan(checkpoint_paths):
        img_list = []
        fixed_noise = torch.randn(64, nz, 1, 1)
        for netG_path in checkpoint_paths:
            loadedG = Generator()
            loadedG.load_state_dict(torch.load(netG_path)["netGmodel"])
            with torch.no_grad():
                fake = loadedG(fixed_noise).detach().cpu()
            img_list.append(vutils.make_grid(fake, padding=2, normalize=True))

        fig = plt.figure(figsize=(8, 8))
        plt.axis("off")
        ims = [[plt.imshow(np.transpose(i, (1, 2, 0)), animated=True)]
               for i in img_list]
        ani = animation.ArtistAnimation(
            fig, ims, interval=1000, repeat_delay=1000, blit=True)
        ani.save("./generated" +str(SA)+"best.gif", writer="imagemagick", dpi=72)
        plt.show()




    # __Trainable_begin__
    class PytorchTrainable(tune.Trainable):
        def setup(self, config):
            use_cuda = config.get("use_gpu") and torch.cuda.is_available()
            self.device = torch.device("cuda" if use_cuda else "cpu")
            self.netD = Discriminator().to(self.device)
            self.netD.apply(weights_init)
            self.netG = Generator().to(self.device)
            self.netG.apply(weights_init)
            self.criterion = nn.BCELoss()    
            self.optimizerD =optim.Adam(
            self.netD.parameters(),
            lr=10**-(config["netD_lr"]),
            betas=(10**-(config["netD_B"]), 0.999),
            weight_decay=10**-(config["weight_decay1"]))
                              
            self.optimizerG =optim.Adam(
            self.netG.parameters(),
            lr=10**-(config["netG_lr"]),
            betas=(10**-(config["netG_B"]), 0.999),
            weight_decay=10**-(config["weight_decay2"]))


            with FileLock(os.path.expanduser("~/.data.lock")):
                self.dataloader = get_data_loader()
            self.mnist_model_ref = c["mnist_model_ref"]

        def step(self):
            lossG, lossD, is_score = train(self.netD, self.netG, self.optimizerG,
                                           self.optimizerD, self.criterion,
                                           self.dataloader, self._iteration,
                                           self.device, self.mnist_model_ref)
            return {"lossg": lossG, "lossd": lossD, "is_score": is_score}

        def save_checkpoint(self, checkpoint_dir):
            path = os.path.join(checkpoint_dir, "checkpoint")
            torch.save({
                "netDmodel": self.netD.state_dict(),
                "netGmodel": self.netG.state_dict(),
                "optimD": self.optimizerD.state_dict(),
                "optimG": self.optimizerG.state_dict(),
            }, path)

            return checkpoint_dir

        def load_checkpoint(self, checkpoint_dir):
            path = os.path.join(checkpoint_dir, "checkpoint")
            checkpoint = torch.load(path)
            self.netD.load_state_dict(checkpoint["netDmodel"])
            self.netG.load_state_dict(checkpoint["netGmodel"])
            self.optimizerD.load_state_dict(checkpoint["optimD"])
            self.optimizerG.load_state_dict(checkpoint["optimG"])

        def reset_config(self, new_config):
            if "netD_lr" in new_config:
                for param_group in self.optimizerD.param_groups:
                    param_group["lr"] = 10**-(new_config["netD_lr"])
            if "netG_lr" in new_config:
                for param_group in self.optimizerG.param_groups:
                    param_group["lr"] = 10**-(new_config["netG_lr"])
            if "netD_B" in new_config:
                for param_group in self.optimizerD.param_groups:
                    param_group["betas"] = (1 - 10**-(new_config["netD_B"]),0.999)
            if "netG_B" in new_config:
                for param_group in self.optimizerG.param_groups:
                    param_group["betas"] = (1 - 10**-(new_config["netG_B"]),0.999)
            if "weight_decay1" in new_config:
                for param_group in self.optimizerD.param_groups:
                    param_group["weight_decay"] = 10**-(new_config["weight_decay1"])
            if "weight_decay2" in new_config:
                for param_group in self.optimizerG.param_groups:
                    param_group["weight_decay"] = 10**-(new_config["weight_decay2"])
            self.config = new_config
            return True

        def _export_model(self, export_formats, export_dir):
            if export_formats == [ExportFormat.MODEL]:
                path = os.path.join(export_dir, "exported_models")
                torch.save({
                    "netDmodel": self.netD.state_dict(),
                    "netGmodel": self.netG.state_dict()
                }, path)
                return {ExportFormat.MODEL: path}
            else:
                raise ValueError("unexpected formats: " + str(export_formats))



    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--smoke-test", action="store_true", help="Finish quickly for testing")
    args, _ = parser.parse_known_args()
    import urllib.request
    # Download a pre-trained MNIST model for inception score calculation.
    # This is a tiny model (<100kb).
    if not os.path.exists(MODEL_PATH):
        print("downloading model")
        os.makedirs(os.path.dirname(MODEL_PATH), exist_ok=True)
        urllib.request.urlretrieve(
            "https://github.com/ray-project/ray/raw/master/python/ray/tune/"
            "examples/pbt_dcgan_mnist/mnist_cnn.pt", MODEL_PATH)

    dataloader = get_data_loader()
    if not args.smoke_test:
        plot_images(dataloader)

    # load the pretrained mnist classification model for inception_score
    mnist_cnn = Net()
    mnist_cnn.load_state_dict(torch.load(MODEL_PATH))
    mnist_cnn.eval()
    mnist_model_ref = ray.put(mnist_cnn)

    # __tune_begin__
    scheduler = PopulationBasedTraining(
        time_attr="training_iteration",
        perturbation_interval=5,
        hyperparam_mutations={
            # distribution for resampling
            "netG_lr": lambda: np.random.uniform(1, 9),
            "netD_lr": lambda: np.random.uniform(1, 9),
            "netD_B": lambda: np.random.uniform(0, 2),
            "netD_B": lambda: np.random.uniform(0, 2),
            "weight_decay1":lambda: np.random.uniform(3, 8),
            "weight_decay2":lambda: np.random.uniform(3, 8)
        })
    from ray.tune.schedulers.pb2 import PB2

    scheduler = PB2(
    time_attr="training_iteration",
    perturbation_interval=3,
    hyperparam_bounds={
        # distribution for resampling
            "netG_lr": [1, 9],
            "netD_lr": [1, 9],
            "netD_B": [0, 2],
            "netD_B": [0, 2],
            "weight_decay1":[3, 8],
            "weight_decay2":[3, 8]
    }) 
    
    c={"mnist_model_ref" : mnist_model_ref}


    experiment_metrics= dict(metric="is_score",
        mode="max")

 
    
    if(SA==0):
        algo = HyperOptSearch(**experiment_metrics)
        name="hypher"

    if(SA==1):
        algo =   BayesOptSearch(**experiment_metrics) 
        name="Bayes"
        
    if(SA==2):
        algo = AxSearch(
            max_concurrent=2, #was working with 2
            **experiment_metrics
        )
        name="AX"

        
        
    if(SA==3):
        algo = NevergradSearch(
            optimizer=ng.optimizers.CMA,**experiment_metrics
            # space=space,  # If you want to set the space manually
            )
        algo = ConcurrencyLimiter(algo, max_concurrent=8)
        name="NG"

        
     
    if(SA==4):
        algo = NevergradSearch(
            optimizer=ng.optimizers.TwoPointsDE,**experiment_metrics
            # space=space,  # If you want to set the space manually
            )
        algo = ConcurrencyLimiter(algo, max_concurrent=8)
        name="NG2DE"

        
        
    if(SA==5):
        algo = NevergradSearch(
        optimizer=ng.optimizers.RandomSearch,**experiment_metrics
        # space=space,  # If you want to set the space manually
        )
        algo = ConcurrencyLimiter(algo, max_concurrent=8)
        name="random"
                
    if(SA==6):
        dim_dict = {
        "netG_lr": (ValueType.CONTINUOUS, [0, 0.1], 1e-2),
        "netD_lr": (ValueType.CONTINUOUS, [0, 0.1], 1e-2)
        }
        algo = ZOOptSearch(
        algo="Asracos",  # only support Asracos currently
        #dim_dict=dim_dict,
      #  dim_dict=dim_dict,

        budget=100,
            **experiment_metrics
        #dim_dict=dim_dict,
       #     **zoopt_search_config,
        )
        name="zoo"


  #  if(SA!=4):
    #scheduler = MedianStoppingRule()
    #AsyncHyperBandScheduler()
        
        
    os.makedirs(os.path.dirname(MODEL_PATH), exist_ok=True)

    tune_kwargs = {
    "num_samples": 20 if args.smoke_test else 20,
      "stop":{
            "training_iteration": 100,
        },  

    "config": {
     "lr1":  tune.uniform(2, 5) #tune.uniform(1e-4, 0.1 ),#,1e-4), #*10
    , "lr2":  tune.uniform(2, 5) #tune.uniform(1e-4, 0.1 ),#,1e-4), #*10
    ,     "weight_decay1":tune.uniform(3, 8)#tune.uniform(1, 5)#,1e-4), #*10 et 0
    ,     "weight_decay2":tune.uniform(3, 8)#tune.uniform(1, 5)#,1e-4), #*10 et 0    
    ,     "beta1":2 #tune.uniform(1, 5),#,1e-4), #*10 et 0
     ,    "beta2":1 #tune.uniform(1, 5),#,1e-4), #*10 et 0
      ,  "adam1":0#tune.uniform(0,1),
      ,  "adam2":0#tune.uniform(0,1),
        }
    }
    def trial_name_id(trial):
        return f"{trial.trainable_name}_{trial.trial_id}"

    analysis = tune.run(
    PytorchTrainable,
    name=name,
    scheduler=scheduler,
    reuse_actors=True,
    search_alg=algo,
    verbose=2,
    checkpoint_at_end=True,
    num_samples=16,
   # export_formats=[ExportFormat.MODEL],
        config =     {
            "netG_lr": tune.uniform(1, 9),
           "netD_lr": tune.uniform(1, 9),
            "netG_B": tune.uniform(0, 3),
           "netD_B": tune.uniform(0, 3),
            "weight_decay1":tune.uniform(3, 8)#tune.uniform(1, 5)#,1e-4), #*10 et 0
    ,     "weight_decay2":tune.uniform(3, 8)
        },      stop={
            "training_iteration": 30,
        },        metric="is_score",
        mode="max"
             ,     loggers=[TestLogger])

    all_trials = analysis.trials
    checkpoint_paths = [
        os.path.join(analysis.get_best_checkpoint(t), "checkpoint")
        for t in all_trials
    ]
    demo_gan(checkpoint_paths)
    

In [15]:
ray.init()



2021-03-12 14:20:46,271	INFO services.py:1092 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '10.1.2.198',
 'raylet_ip_address': '10.1.2.198',
 'redis_address': '10.1.2.198:6379',
 'object_store_address': '/tmp/ray/session_2021-03-12_14-20-45_188076_4892/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-03-12_14-20-45_188076_4892/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-03-12_14-20-45_188076_4892',
 'metrics_export_port': 58687,
 'node_id': '3c1e3902a0cf0a28e93947298b5992a5fd3d89cd'}

In [16]:
from ray.tune.logger import *
supernombre="PB2TRAN";
class TestLogger(tune.logger.Logger):
    def _init(self):
        progress_file = os.path.join("/home/antoine/Projet/NovelTuning/", supernombre+"com.csv")
        self._continuing = os.path.exists(progress_file)
        self._file = open(progress_file, "a")
        self._csv_out = None
    def on_result(self, result):
        tmp = result.copy()
        #if "done" in tmp:
         #   if(tmp["done"] != True):

        if "config" in tmp:
            del tmp["config"]
        result = flatten_dict(tmp, delimiter="/")
        if self._csv_out is None:
            self._csv_out = csv.DictWriter(self._file, result.keys())
            if not self._continuing:
                self._csv_out.writeheader()
        self._csv_out.writerow(
            {k: v
             for k, v in result.items() if k in self._csv_out.fieldnames})
        self._file.flush()
                   
            
GAN_MNIST(5)

    

2021-03-12 14:20:48,191	WARNING logger.py:328 -- JsonLogger not provided. The ExperimentAnalysis tool is disabled.


Trial name,status,loc,netD_B,netD_lr,netG_B,netG_lr,weight_decay1,weight_decay2
PytorchTrainable_c2551280,RUNNING,,0.255619,1.50252,0.77397,6.03856,4.55235,5.15503


(pid=12694)  /home/antoine/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:3121: UserWarning:Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
(pid=12694)  /home/antoine/anaconda3/lib/python3.7/site-packages/ray/workers/default_worker.py:254: UserWarning:Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
(pid=12698)  /home/antoine/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:3121: UserWarning:Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
(pid=12698)  /home/antoine/anaconda3/lib/python3.7/site-packages/ray/workers/default_worker.py:254: UserWarning:Implicit dimension choice for softma

Result for PytorchTrainable_c25fa8bc:
  date: 2021-03-12_14-20-53
  done: false
  experiment_id: e58aa7aa02d343fb8e27d9398c98cb85
  experiment_tag: 2_netD_B=0.76543,netD_lr=5.1371,netG_B=0.97797,netG_lr=4.1861,weight_decay1=7.0892,weight_decay2=5.104
  hostname: antoine-HP-Spectre-Laptop-13-af0xx
  is_score: 1.0000000000000004
  iterations_since_restore: 1
  lossd: 1.5880701541900635
  lossg: 0.6416276693344116
  node_ip: 10.1.2.198
  pid: 12694
  time_since_restore: 2.5769245624542236
  time_this_iter_s: 2.5769245624542236
  time_total_s: 2.5769245624542236
  timestamp: 1615555253
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: c25fa8bc
  


Trial name,status,loc,netD_B,netD_lr,netG_B,netG_lr,weight_decay1,weight_decay2,iter,total time (s),lossg,lossd,is_score
PytorchTrainable_c2551280,RUNNING,,0.255619,1.50252,0.77397,6.03856,4.55235,5.15503,,,,,
PytorchTrainable_c25fa8bc,RUNNING,10.1.2.198:12694,0.765429,5.13705,0.977972,4.18609,7.08923,5.10404,1,2.57692,0.641628,1.58807,1
PytorchTrainable_c262ef68,RUNNING,,0.956785,2.53445,0.953325,4.72262,6.4508,5.73415,,,,,
PytorchTrainable_c265fbd6,RUNNING,,1.20648,7.14941,0.644432,5.30553,5.86367,5.1391,,,,,
PytorchTrainable_c269dd1e,RUNNING,,1.14477,4.66077,0.600882,3.64255,6.05619,6.49979,,,,,
PytorchTrainable_c26d0d54,RUNNING,,0.846731,4.60001,1.4341,4.48102,6.86691,5.59992,,,,,
PytorchTrainable_c270956e,RUNNING,,1.04503,7.79686,1.35942,6.29852,7.58801,6.4283,,,,,
PytorchTrainable_c27a58f6,RUNNING,,0.833915,6.13565,0.961426,2.46292,5.60151,5.2504,,,,,


...

Result for PytorchTrainable_f9d8bbda:
  date: 2021-03-12_14-23-11
  done: false
  experiment_id: 2b859fb4ac41446ab1a1e0f6a7ace33c
  experiment_tag: 16_netD_B=0.084372,netD_lr=4.0879,netG_B=0.91287,netG_lr=6.034,weight_decay1=4.5573,weight_decay2=4.585
  hostname: antoine-HP-Spectre-Laptop-13-af0xx
  is_score: 1.0000000000000004
  iterations_since_restore: 27
  lossd: 0.005663279443979263
  lossg: 6.019207000732422
  node_ip: 10.1.2.198
  pid: 14742
  time_since_restore: 43.0130558013916
  time_this_iter_s: 2.149265766143799
  time_total_s: 43.0130558013916
  timestamp: 1615555391
  timesteps_since_restore: 0
  training_iteration: 27
  trial_id: f9d8bbda
  


Trial name,status,loc,netD_B,netD_lr,netG_B,netG_lr,weight_decay1,weight_decay2,iter,total time (s),lossg,lossd,is_score
PytorchTrainable_f9d8bbda,RUNNING,10.1.2.198:14742,0.084372,4.08785,0.912872,6.03397,4.55731,4.58495,27,43.0131,6.01921,0.00566328,1
PytorchTrainable_c2551280,ERROR,,1.04503,7.79686,1.35942,6.29852,7.58801,6.4283,5,19.0173,0.789408,1.6445,1
PytorchTrainable_c25fa8bc,ERROR,,1.04503,7.79686,1.35942,6.29852,7.58801,6.4283,5,18.9261,0.78418,1.6213,1
PytorchTrainable_c262ef68,ERROR,,0.956785,2.53445,0.953325,4.72262,6.4508,5.73415,8,33.3747,8.94224,0.00106892,1
PytorchTrainable_c265fbd6,ERROR,,1.20648,7.14941,0.644432,5.30553,5.86367,5.1391,8,32.7041,0.616088,1.48325,1
PytorchTrainable_c269dd1e,ERROR,,1.14477,4.66077,0.600882,3.64255,6.05619,6.49979,11,45.3075,0.180798,2.50897,1
PytorchTrainable_c26d0d54,ERROR,,0.846731,4.60001,1.4341,4.48102,6.86691,5.59992,11,45.5197,1.01925,0.690928,1
PytorchTrainable_c270956e,ERROR,,1.04503,7.79686,1.35942,6.29852,7.58801,6.4283,11,45.5906,0.830612,1.62145,1
PytorchTrainable_c27a58f6,ERROR,,1.00403,6.27028,1.2544,3.46235,6.68186,4.95232,5,21.5754,0.0163755,5.28184,1
PytorchTrainable_d97e8e96,ERROR,,1.00403,6.27028,1.2544,3.46235,6.68186,4.95232,5,20.312,0.0167838,5.2185,1


Result for PytorchTrainable_f9d8bbda:
  date: 2021-03-12_14-23-15
  done: true
  experiment_id: 2b859fb4ac41446ab1a1e0f6a7ace33c
  experiment_tag: 16_netD_B=0.084372,netD_lr=4.0879,netG_B=0.91287,netG_lr=6.034,weight_decay1=4.5573,weight_decay2=4.585
  hostname: antoine-HP-Spectre-Laptop-13-af0xx
  is_score: 1.0000000000000004
  iterations_since_restore: 30
  lossd: 0.005500889848917723
  lossg: 6.074499607086182
  node_ip: 10.1.2.198
  pid: 14742
  time_since_restore: 46.720534801483154
  time_this_iter_s: 1.2115044593811035
  time_total_s: 46.720534801483154
  timestamp: 1615555395
  timesteps_since_restore: 0
  training_iteration: 30
  trial_id: f9d8bbda
  


Trial name,status,loc,netD_B,netD_lr,netG_B,netG_lr,weight_decay1,weight_decay2,iter,total time (s),lossg,lossd,is_score
PytorchTrainable_f9d8bbda,TERMINATED,,0.084372,4.08785,0.912872,6.03397,4.55731,4.58495,30,46.7205,6.0745,0.00550089,1
PytorchTrainable_c2551280,ERROR,,1.04503,7.79686,1.35942,6.29852,7.58801,6.4283,5,19.0173,0.789408,1.6445,1
PytorchTrainable_c25fa8bc,ERROR,,1.04503,7.79686,1.35942,6.29852,7.58801,6.4283,5,18.9261,0.78418,1.6213,1
PytorchTrainable_c262ef68,ERROR,,0.956785,2.53445,0.953325,4.72262,6.4508,5.73415,8,33.3747,8.94224,0.00106892,1
PytorchTrainable_c265fbd6,ERROR,,1.20648,7.14941,0.644432,5.30553,5.86367,5.1391,8,32.7041,0.616088,1.48325,1
PytorchTrainable_c269dd1e,ERROR,,1.14477,4.66077,0.600882,3.64255,6.05619,6.49979,11,45.3075,0.180798,2.50897,1
PytorchTrainable_c26d0d54,ERROR,,0.846731,4.60001,1.4341,4.48102,6.86691,5.59992,11,45.5197,1.01925,0.690928,1
PytorchTrainable_c270956e,ERROR,,1.04503,7.79686,1.35942,6.29852,7.58801,6.4283,11,45.5906,0.830612,1.62145,1
PytorchTrainable_c27a58f6,ERROR,,1.00403,6.27028,1.2544,3.46235,6.68186,4.95232,5,21.5754,0.0163755,5.28184,1
PytorchTrainable_d97e8e96,ERROR,,1.00403,6.27028,1.2544,3.46235,6.68186,4.95232,5,20.312,0.0167838,5.2185,1


TuneError: ('Trials did not complete', [PytorchTrainable_c2551280, PytorchTrainable_c25fa8bc, PytorchTrainable_c262ef68, PytorchTrainable_c265fbd6, PytorchTrainable_c269dd1e, PytorchTrainable_c26d0d54, PytorchTrainable_c270956e, PytorchTrainable_c27a58f6, PytorchTrainable_d97e8e96, PytorchTrainable_d9b862b0, PytorchTrainable_da6addc8, PytorchTrainable_db41329c, PytorchTrainable_e241e8fc, PytorchTrainable_e26f648a, PytorchTrainable_e2d5a448])

In [9]:
# IN THIS MODULE: IMPORTS, CNN, TRAIN, TEST, MNIS_FUNCTION, SPACE

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
from ray import tune
from ray.tune.schedulers import ASHAScheduler
from hyperopt import hp
from ray.tune.suggest.hyperopt import HyperOptSearch
from ray.tune.suggest.skopt import SkOptSearch
from ray import tune
from ray.tune.suggest.bayesopt import BayesOptSearch
import time
import ray
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest.ax import AxSearch
import argparse
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest.nevergrad import NevergradSearch
import nevergrad as ng
import json
import os
from ray.tune import Trainable
from ray.tune.schedulers.hb_bohb import HyperBandForBOHB
from ray.tune.suggest.bohb import TuneBOHB
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest.dragonfly import DragonflySearch
from ray.tune.suggest.zoopt import ZOOptSearch
from ray.tune.schedulers import AsyncHyperBandScheduler
from zoopt import ValueType
import torch
# IN THIS MODULE: IMPORTS, CNN, TRAIN, TEST, MNIS_FUNCTION, SPACE

import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
from ray import tune
from ray.tune.schedulers import ASHAScheduler 
from ray.tune.schedulers import MedianStoppingRule

from hyperopt import hp
from ray.tune.suggest.hyperopt import HyperOptSearch
from ray.tune.suggest.skopt import SkOptSearch
from ray import tune
from ray.tune.suggest.bayesopt import BayesOptSearch
import time
import ray
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest.ax import AxSearch
import argparse
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest.nevergrad import NevergradSearch
import nevergrad as ng
import json
import os
from ray.tune import Trainable
from ray.tune.schedulers.hb_bohb import HyperBandForBOHB
from ray.tune.suggest.bohb import TuneBOHB
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.schedulers import AsyncHyperBandScheduler
#from ray.tune.suggest.dragonfly import DragonflySearch
from ray.tune.suggest.zoopt import ZOOptSearch
from ray.tune.schedulers import AsyncHyperBandScheduler
from zoopt import ValueType
import torch
import adabelief_pytorch
global_checkpoint_period=np.inf

import ray

import os
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np

import ray
from ray import tune
from ray.tune.trial import ExportFormat
from ray.tune.schedulers import PopulationBasedTraining

import argparse
import os
from filelock import FileLock
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import numpy as np
from ray.tune.suggest.bayesopt import BayesOptSearch
from ray.tune.suggest.ax import AxSearch



from torch.autograd import Variable
from torch.nn import functional as F
from scipy.stats import entropy

import matplotlib.pyplot as plt
import matplotlib.animation as animation

# Training parameters
dataroot = ray.utils.get_user_temp_dir() + os.sep
workers = 2
batch_size = 64
image_size = 32

# Number of channels in the training images. For color images this is 3
nc = 1

# Size of z latent vector (i.e. size of generator input)
nz = 100

# Size of feature maps in generator
ngf = 32

# Size of feature maps in discriminator
ndf = 32

# Beta1 hyperparam for Adam optimizers
beta1 = 0.5

# iterations of actual training in each Trainable _train
train_iterations_per_step = 5

MODEL_PATH = os.path.expanduser("~/.ray/models/mnist_cnn.pt")


def get_data_loader():
    dataset = dset.MNIST(
        root=dataroot,
        download=True,
        transform=transforms.Compose([
            transforms.Resize(image_size),
            transforms.ToTensor(),
            transforms.Normalize((0.5, ), (0.5, )),
        ]))

    # Create the dataloader
    dataloader = torch.utils.data.DataLoader(
        dataset, batch_size=batch_size, shuffle=True, num_workers=workers)

    return dataloader


# __GANmodel_begin__
# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm") != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)


# Generator Code
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d(nz, ngf * 4, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh())

    def forward(self, input):
        return self.main(input)


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2), nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4), nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 4, 1, 4, 1, 0, bias=False), nn.Sigmoid())

    def forward(self, input):
        return self.main(input)


# __GANmodel_end__


# __INCEPTION_SCORE_begin__
class Net(nn.Module):
    """
    LeNet for MNist classification, used for inception_score
    """

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)


def inception_score(imgs, mnist_model_ref, batch_size=32, splits=1):
    N = len(imgs)
    dtype = torch.FloatTensor
    dataloader = torch.utils.data.DataLoader(imgs, batch_size=batch_size)
    cm = mnist_model_ref  # Get the mnist model from Ray object store.
    up = nn.Upsample(size=(28, 28), mode="bilinear").type(dtype)

    def get_pred(x):
        x = up(x)
        x = cm(x)
        return F.softmax(x).data.cpu().numpy()

    preds = np.zeros((N, 10))
    for i, batch in enumerate(dataloader, 0):
        batch = batch.type(dtype)
        batchv = Variable(batch)
        batch_size_i = batch.size()[0]
        preds[i * batch_size:i * batch_size + batch_size_i] = get_pred(batchv)

    # Now compute the mean kl-div
    split_scores = []
    for k in range(splits):
        part = preds[k * (N // splits):(k + 1) * (N // splits), :]
        py = np.mean(part, axis=0)
        scores = []
        for i in range(part.shape[0]):
            pyx = part[i, :]
            scores.append(entropy(pyx, py))
        split_scores.append(np.exp(np.mean(scores)))

    return np.mean(split_scores), np.std(split_scores)


# __INCEPTION_SCORE_end__


def train(netD, netG, optimG, optimD, criterion, dataloader, iteration, device,
          mnist_model_ref):
    real_label = 1
    fake_label = 0

    for i, data in enumerate(dataloader, 0):
        if i >= train_iterations_per_step:
            break

        netD.zero_grad()
        real_cpu = data[0].to(device)
        b_size = real_cpu.size(0)
        label = torch.full(
            (b_size, ), real_label, dtype=torch.float, device=device)
        output = netD(real_cpu).view(-1)
        errD_real = criterion(output, label)
        errD_real.backward()
        D_x = output.mean().item()

        noise = torch.randn(b_size, nz, 1, 1, device=device)
        fake = netG(noise)
        label.fill_(fake_label)
        output = netD(fake.detach()).view(-1)
        errD_fake = criterion(output, label)
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        errD = errD_real + errD_fake
        optimD.step()

        netG.zero_grad()
        label.fill_(real_label)
        output = netD(fake).view(-1)
        errG = criterion(output, label)
        errG.backward()
        D_G_z2 = output.mean().item()
        optimG.step()

        is_score, is_std = inception_score(fake, mnist_model_ref)

    return errG.item(), errD.item(), is_score


import copy

# __Trainable_begin__
class PytorchTrainable():
    def __init__(self, config):
        use_cuda = config.get("use_gpu") and torch.cuda.is_available()
        self.device = torch.device("cuda" if use_cuda else "cpu")
        self.netD = Discriminator().to(self.device)
        self.netD.apply(weights_init)
        self.netG = Generator().to(self.device)
        self.netG.apply(weights_init)
        self.criterion = nn.BCELoss()   
        self.iteration = 0
        self.optimizerD =optim.Adam(
            self.netD.parameters(),
            lr=10**-(config["netD_lr"]),
            betas=(1-10**-(config["netD_B"]), 0.999),
            weight_decay=10**-(config["weight_decay1"]))
                              
        self.optimizerG =optim.Adam(
            self.netG.parameters(),
            lr=10**-(config["netG_lr"]),
            betas=(1-10**-(config["netG_B"]), 0.999),
            weight_decay=10**-(config["weight_decay2"]))

        with FileLock(os.path.expanduser("~/.data.lock")):
            self.dataloader = get_data_loader()
        self.mnist_model_ref = mnist_cnn

    def step(self):
        lossG, lossD, is_score = train(self.netD, self.netG, self.optimizerG,
                                       self.optimizerD, self.criterion,
                                       self.dataloader, self.iteration,
                                       self.device, self.mnist_model_ref)
        self.iteration+=1
        return is_score #{"lossg": lossG, "lossd": lossD, "is_score": is_score}


    def adapt(self, new_config):

        temp = copy.deepcopy(self)

        if "netD_lr" in new_config:
            for param_group in temp.optimizerD.param_groups:
                param_group["lr"] = 10**-(new_config["netD_lr"])
        if "netG_lr" in new_config:
            for param_group in temp.optimizerG.param_groups:
                param_group["lr"] = 10**-(new_config["netG_lr"])
        if "netD_B" in new_config:
            for param_group in temp.optimizerD.param_groups:
                param_group["betas"] = (1-10**-(new_config["netD_B"]),0.999)
        if "netG_B" in new_config:
            for param_group in temp.optimizerG.param_groups:
                param_group["betas"] = (1-10**-(new_config["netG_B"]),0.999)
        if "weight_decay1" in new_config:
            for param_group in temp.optimizerD.param_groups:
                param_group["weight_decay"] = 10**-(new_config["weight_decay1"])
        if "weight_decay2" in new_config:
            for param_group in temp.optimizerG.param_groups:
                param_group["weight_decay"] = 10**-(new_config["weight_decay2"])
               

        
        return temp

        #temp = copy.deepcopy(self)
        #for key, value in config.items():
        #    temp.config[key] = value
        #config = temp.config

        #temp.model.adapt(config.get("droupout_prob", 0.5))


parser = argparse.ArgumentParser()
parser.add_argument(
    "--smoke-test", action="store_true", help="Finish quickly for testing")
args, _ = parser.parse_known_args()
import urllib.request
# Download a pre-trained MNIST model for inception score calculation.
# This is a tiny model (<100kb).
if not os.path.exists(MODEL_PATH):
    print("downloading model")
    os.makedirs(os.path.dirname(MODEL_PATH), exist_ok=True)
    urllib.request.urlretrieve(
        "https://github.com/ray-project/ray/raw/master/python/ray/tune/"
        "examples/pbt_dcgan_mnist/mnist_cnn.pt", MODEL_PATH)

dataloader = get_data_loader()
#if not args.smoke_test:
#    plot_images(dataloader)

# load the pretrained mnist classification model for inception_score
mnist_cnn = Net()
mnist_cnn.load_state_dict(torch.load(MODEL_PATH))
mnist_cnn.eval()
#  mnist_model_ref = ray.put(mnist_cnn)


from ray.tune.schedulers.pb2 import PB2

scheduler = PB2(
time_attr="training_iteration",
perturbation_interval=1,
hyperparam_bounds={
    # distribution for resampling
        "netG_lr": [1, 9],
        "netD_lr": [1, 9],
        "netD_B": [0, 2],
        "netD_B": [0, 2],
        "weight_decay1":[3, 8],
        "weight_decay2":[3, 8]
}) 

#    c={"mnist_model_ref" : mnist_model_ref}


experiment_metrics= dict(metric="is_score",
    mode="max")





In [2]:


class Parent():
    def __init__(self,hyperspace,configuration, model, loss):
        
        self.hyperspace = hyperspace
        self.configuration_list = [configuration] 
        self.loss_list = [np.array(loss)] 
        self.model = model
    
    def update(self,configuration,loss, model):
        self.configuration_list = np.append(self.configuration_list,configuration) 
        self.loss_list = np.append(self.loss_list,loss)
        self.model = model
    
    def get_hyperspace(self):
        return self.hyperspace

    def get_model(self):
        return self.model
    
    def get_loss(self):
        return self.loss_list
    

In [11]:
import copy


class Scheduler():
    def __init__(self, model, num_iteration, num_config,
                 oracle):
        #Oracle manages the Bayesian optimization
        self.oracle = oracle
        self.iteration = num_iteration
        self.num_config = num_config 
        self.sqrt_config =   math.floor(math.sqrt(num_config)) # math.ceil(num_config/5) #
        
        #self.h is for the m "h" used at every loop, h is a configuration from the search space
        self.h = np.repeat({},num_config) 
        
        #self.out is for storing the result of the algorithm, ie all "h" from all iterations
        #from all sqrt(m) best models per iterations.
        self.out = np.zeros((num_iteration,self.sqrt_config))
        
        #self.hyperspaces is for storing the sqrt(m) hyperspaces used by the algorithm
        self.hyperspaces = np.zeros(self.sqrt_config)
        
        self.plot = np.zeros(num_iteration)

        
        #self.model is the m model that will explore new hyperspace points at every iterations
        self.models = np.repeat(model,num_config)
        
        #self.parents is the sqrt(m) best model from last iteration
        self.parents = np.repeat(model,self.sqrt_config)

        #self.losses remembers the performances of all m models at one iteration to decide which ones are the sqrt(m) best from self.models.
        self.losses = np.zeros(num_config)
        
        self.k = [0] # c'est pour avoir un pointeur sur k, c'est pas plus que O(sqrt)-paralélisable  pour le moment du coup.
    
    def initialisation(self):
        num_config = self.num_config
        extended_Hyperspace = Trials()
        fmin_objective = partial(test_function, models=self.models,h=self.h,losses=self.losses,parent_model=self.models, k_f = self.k,iteration = 0)
        self.oracle.compute_Batch(extended_Hyperspace ,num_config , 0 ,fmin_objective)
            
        indexes = np.argsort(self.losses)     
        self.out[0] = (self.losses[indexes])[0:self.sqrt_config]
        self.hyperspaces = np.repeat(extended_Hyperspace,self.sqrt_config)    
        self.parents = np.array([Parent(copy.deepcopy(extended_Hyperspace),(self.h[indexes])[i], (self.models[indexes])[i],(self.losses[indexes])[i])  
                                 for i in range(self.sqrt_config) ])         
        self.plot[0] = self.losses[indexes][0]
        
    def loop(self):
        sqrt_config = self.sqrt_config
        iteration = self.iteration
        for i in range(1,iteration):
            
            self.k[0] = 0

            for j in range(sqrt_config):
                parent = self.parents[j]
                extended_Hyperspace = parent.get_hyperspace()
                print("\n loss of parent " + str(parent.get_loss()[-1]) )
                print("\n loss " + str(parent.get_loss()))
                
    
                fmin_objective = partial(test_function, models=self.models,h=self.h,losses=self.losses,parent_model=parent.get_model(), k_f = self.k,iteration = i)
                self.oracle.Repeat_good(extended_Hyperspace ,i ,fmin_objective,parent.configuration_list[-1])
                self.oracle.compute_Batch(extended_Hyperspace ,int(self.num_config/sqrt_config) -1 , i ,fmin_objective)
                   
            indexes = np.argsort(self.losses)     
            parent_idx = indexes[:sqrt_config]
            print(self.losses)
            print(indexes)
 
            temp = np.empty(self.sqrt_config, dtype=Parent)
            for a,x in enumerate(parent_idx):
                x = int(x)
                temp[a] = copy.deepcopy(self.parents[math.floor(x/self.num_config*sqrt_config)])
                temp[a].update(self.h[x], self.losses[x], self.models[x])
            self.parents = temp
            self.plot[i] = self.losses[parent_idx][0]


    
    
CONFIGURATION = 100
ITERATIONS = 3
config= {
     "netG_lr": hp.uniform("netG_lr",1,9)
    , "netD_lr": hp.uniform("netD_lr",1,9)
      ,   "netG_B": hp.uniform("netG_B",0,3),
    "netD_B" : hp.uniform("netD_B",0, 3),
    "weight_decay1" : hp.uniform("weight_decay1",3, 8),
    "weight_decay2" : hp.uniform("weight_decay2",3, 8)}

model = PytorchTrainable
oracle = Oracle(config)
scheduler = Scheduler(
    model,
    ITERATIONS,
    CONFIGURATION,
    oracle) 

scheduler.initialisation()     
scheduler.loop()     


itération 0.0                                          
netD_B 0.38660832249559474                             
netD_lr 7.298388096483894                              
netG_B 0.9670771214934172                              
netG_lr 7.418760770463007                              
weight_decay1 7.501755958563166                        
weight_decay2 7.265052676447382                        
  0%|          | 0/100 [00:00<?, ?trial/s, best loss=?]

INFO:filelock:Lock 140443476258960 acquired on /home/antoine/.data.lock
INFO:filelock:Lock 140443476258960 released on /home/antoine/.data.lock
/home/antoine/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:253: UserWarning:

Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.




accuracy, 1.0000000000000029                           

itération -0.0                                                                    
netD_B 0.1262179566553448                                                         
netD_lr 8.141507488515586                                                         
netG_B 0.6421548035772979                                                         
netG_lr 8.109210702932003                                                         
weight_decay1 7.973881950921562                                                   
weight_decay2 7.973096699893967                                                   
  1%|          | 1/100 [00:03<05:50,  3.54s/trial, best loss: -1.0000000000000029]

INFO:filelock:Lock 140444033663888 acquired on /home/antoine/.data.lock
INFO:filelock:Lock 140444033663888 released on /home/antoine/.data.lock


accuracy, 1.0000000000000029                                                      

itération 0.0                                                                     
netD_B 2.3668860716115905                                                         
netD_lr 2.736955021175844                                                         
netG_B 2.9816843007040053                                                         
netG_lr 1.9051212964182231                                                        
weight_decay1 4.127874782947522                                                   
weight_decay2 4.436464217436043                                                   
  2%|▏         | 2/100 [00:07<05:51,  3.58s/trial, best loss: -1.0000000000000029]

...

itération 1.0                                                                       
netD_B 1.5205844343559258                                                           
netD_lr 5.866060138143053                                                           
netG_B 1.5917835037798465                                                           
netG_lr 2.301636280445578                                                           
weight_decay1 4.435137813501811                                                     
weight_decay2 7.478383479024645                                                     
accuracy, 1.2728406364488354                                                        

itération 1.0                                                                       
netD_B 1.5532994674334748                                                           
netD_lr 7.251490360150135                                                           
netG_B 1.6058182380953516                                       

itération 1.0                                            
netD_B 1.4638644111870551                                          
netD_lr 1.463034150032986                                          
netG_B 1.9387830265701667                                          
netG_lr 2.0569640176806825                                         
weight_decay1 5.038479923479032                                    
weight_decay2 4.782422571633792                                    
accuracy, 1.2287869027738771                                       

100%|██████████| 101/101 [00:04<00:00, 20.72trial/s, best loss: -1.3135538937560618]
itération 1.0                                            
netD_B 0.28986299065450516                                         
netD_lr 4.298843691028078                                          
netG_B 0.7982860375184155                                          
netG_lr 1.153020933216717                                          
weight_decay1 6.088472239161774                   

netG_B 0.63233007733509                                                             
netG_lr 2.3756875310867964                                                          
weight_decay1 5.339176770212221                                                     
weight_decay2 4.514323429728801                                                     
accuracy, 1.1051665959542758                                                        

itération 1.0                                                                       
netD_B 0.32319760721550006                                                          
netD_lr 3.1744938384861285                                                          
netG_B 2.0923435810825706                                                           
netG_lr 4.803243363939539                                                           
weight_decay1 3.5679297476712164                                                    
weight_decay2 6.28108035689994                                  

netG_B 0.6714320378184715                                                           
netG_lr 5.385274975288631                                                           
weight_decay1 3.59528835680481                                                      
weight_decay2 6.131652027446925                                                     
accuracy, 1.0308215827019367                                                        

itération 1.0                                                                       
netD_B 2.782568987649899                                                            
netD_lr 2.8765070184340384                                                          
netG_B 1.1672954497385137                                                           
netG_lr 1.5020846321016723                                                          
weight_decay1 5.022537440008581                                                     
weight_decay2 4.385689137018139                                 

KeyboardInterrupt: 

In [4]:
import math
from hyperopt import hp, fmin, tpe, Trials
from functools import *
# Oracle (Paul) TODO
class Oracle():
    def __init__(self, searchspace ):
        #self.hyperspace is the original (input) searchspace
        self.searchspace = searchspace

    def Repeat_good(self,trials, iteration,function,configuration): #add space
        space = copy.deepcopy(configuration)
        for k,v in configuration.items():
            space[k] =  hp.uniform(k,-1e-10+v,v + 1e-10) 

        curr_eval = getattr(trials,'_ids')
        if curr_eval == set():
            curr_eval = 0
        else:
            curr_eval = max(curr_eval) +1
        space["itération"] =  hp.quniform("itération",-.5+iteration,.5+iteration, 1) 
        fmin(function, space, algo=partial(tpe.suggest, n_startup_jobs=1), max_evals=curr_eval
+1, trials=trials)
        
    def compute_Once(self,trials, iteration,function): #add space
        space = copy.deepcopy(self.searchspace)
        curr_eval = getattr(trials,'_ids')
        if curr_eval == set():
            curr_eval = 0
        else:
            curr_eval = max(curr_eval) +1
        space["itération"] =  hp.quniform("itération",-.5+iteration,.5+iteration, 1) 
        fmin(function, space, algo=partial(tpe.suggest, n_startup_jobs=1), max_evals=curr_eval
+1, trials=trials)
        
        
    def compute_Batch(self,trials, nb_eval, iteration,function): #add space
        space = copy.deepcopy(self.searchspace)
        curr_eval = getattr(trials,'_ids')
        if curr_eval == set():
            curr_eval = 0
        else:
            curr_eval = max(curr_eval) +1
            
        space["itération"] =  hp.quniform("itération",-.5+iteration,.5+iteration, 1) 
        fmin(function, space, algo=partial(tpe.suggest, n_startup_jobs=1), max_evals=curr_eval
+nb_eval, trials=trials)
    


In [6]:
def test_function(x,models,h,losses, parent_model,k_f,iteration):
  
    for key, value in x.items():
            print(key + " "+str(x[key]))

    

    if (isinstance(k_f,list)):
            k=k_f[0]
            Islist = True 
    else:
            k = k_f
            Islist = False
        
    if iteration == 0:
     #   k = k_f[0]
        models[k] = parent_model[k](x)
   #     k_f[0] += 1
    else:      
       # k = k_f
        models[k] = parent_model.adapt(x)
       # k_f[0] += 1
    if(Islist):
        k_f[0] += 1
    


    h[k] = x
    loss = models[k].step()
    loss = models[k].step()
    loss = models[k].step()
    losses[k] = -loss
    print("accuracy, " + str(loss) + "\n")
    return -loss

In [ ]:

from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST, FashionMNIST, CIFAR10, STL10
import torchvision.transforms as transforms
import os
from ray.tune.suggest.hyperopt import HyperOptSearch
from ray.tune.schedulers import ASHAScheduler
from ray.tune.schedulers import PopulationBasedTraining
from ray.tune.suggest import ConcurrencyLimiter
from ray import tune
import gc
import numpy as np
from ray.tune.schedulers.pb2 import PB2


#train_loader = torch.utils.data.DataLoader(
#    datasets.MNIST('../data', train=True, download=True,
#                   transform=transforms.ToTensor()),
#    batch_size=args.batch_size, shuffle=True, **kwargs)
#test_loader = torch.utils.data.DataLoader(
#    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
#batch_size=args.batch_size, shuffle=True, **kwargs)

class VAE(nn.Module):
    def __init__(self,config):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        super(VAE, self).__init__()
        dim1= int(config.get("d1", 0)) #36*12*4*3
        dim2=int(config.get("d2", 0)) #36*12*3
        dim3=int(config.get("d3", 0)) #20*27
        dim4=int(config.get("d4", 0)) #dim2
        dim5=int(config.get("d5", 0)) #dim1

        print("gang")
        self.lf=image_L*image_L*3
        print("endan")

        self.fc1 = nn.Linear(self.lf, dim1)
        print("endan")

        self.fc12 = nn.Linear(dim1, dim2)
        print("endan")

        self.fc21 = nn.Linear(dim2, dim3)
        print("endan")
        self.fc22 = nn.Linear(dim2, dim3)
        print("endan")

        self.fc23 = nn.Linear(dim3,dim4)

        print("endan")
        self.fc3 = nn.Linear(dim4, dim5)
        print("endan")

        self.fc4 = nn.Linear(dim5, self.lf)
        print("endan")

    def encode(self, x):
        h1 = F.relu(self.fc1(x.view(-1, self.lf)))
        h1 = F.relu(self.fc12(h1))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        h3 = F.relu(self.fc23(z))
        h3 = F.relu(self.fc3(h3))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

class PytorchTrainable(tune.Trainable):

    def setup(self, config):
        use_cuda = torch.cuda.is_available()
        self.device = torch.device("cuda" if use_cuda else "cpu")

        #self.netD = VAES(channel_in=3).to(self.device)
        self.vae_net = VAES(channel_in = 3).to(self.device)


        lr=10**-(config.get("lr", 0))
        beta1=1-10**-(config.get("b1", 0))
        beta2=1-10**-(config.get("b2", 0))
        eps=10**-(config.get("eps", 0))
        weight_decay=10**-(config.get("wd", 0))
        self.optimizerD = optim.Adam(self.vae_net.parameters(),
                                  lr=lr, betas=(beta1, beta2),
                               eps=eps, weight_decay=weight_decay)
        self.dimbotle=128*4
        globalinc=0

        self.train_loader=  get_pinned_object(X_id)
        self.test_loader=  get_pinned_object(Y_id)




        #kwargs = {'num_workers': 1, 'pin_memory': False} if args.cuda else {}
       # self.train_loader = torch.utils.data.DataLoader(
        #    datasets.STL10('/content/drive/My Drive/', split="unlabeled", download=True,
        #                   transform=transforms.ToTensor()),
        #        batch_size=args.batch_size, shuffle=True, **kwargs)
        #self.test_loader = torch.utils.data.DataLoader(
        #    datasets.STL10('/content/drive/My Drive/', split="test", transform=transforms.ToTensor(),download=True),
        #batch_size=args.batch_size, shuffle=True, **kwargs)


    def step(self):
        device = torch.device("cuda" if  torch.cuda.is_available() else "cpu")
        train(self.vae_net,self.optimizerD,self.train_loader)
        loss=test(self.vae_net,self.test_loader)
        with torch.no_grad():
            sample = torch.randn(64,self.dimbotle).to(device)
            sample = self.vae_net.decoder(sample).cpu()
            save_image(sample.view(64, 3, cst, cst),path +'sample_' + str(os.getpid()) + '.png')

    
        return {"loss": loss}

    def save_checkpoint(self, checkpoint_dir):
        path = os.path.join(checkpoint_dir, "checkpoint")
        torch.save({
            "netDmodel": self.vae_net.state_dict(),
            "optimD": self.optimizerD.state_dict(),

        }, path)

        return checkpoint_dir

    def load_checkpoint(self, checkpoint_dir):
        path = os.path.join(checkpoint_dir, "checkpoint")
        checkpoint = torch.load(path)
        self.vae_net.load_state_dict(checkpoint["netDmodel"])
        self.optimizerD.load_state_dict(checkpoint["optimD"])

    def reset_config(self, new_config):
        if "lr" in new_config:
            for param_group in self.optimizerD.param_groups:
                param_group["lr"] = 10**-(new_config.get("lr", 0))
        if "b1" in new_config:
            for param_group in self.optimizerD.param_groups:
                param_group["betas"] = (
                    1-10**-(new_config.get("b1", 0)),1-10**-(new_config.get("b2", 0)))
        if "wd" in new_config:
            for param_group in self.optimizerD.param_groups:
                param_group["weight_decay"] = 10**-(new_config.get("wd", 0))
        if "eps" in new_config:
            for param_group in self.optimizerD.param_groups:
                param_group["eps"] = 10**-(new_config.get("eps", 0))
        self.config = new_config
        return True



#  model = VAE().to(device)
#optimizer = adabelief_pytorch.AdaBelief(model.parameters(), lr=1e-3)





# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy_with_logits(recon_x, x.view(-1, cst*cst*3), reduction='sum')

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD


def train(model,optimizer,train_loader):
    model.train()
    train_loss = 0
    device = torch.device("cuda" if  torch.cuda.is_available() else "cpu")
    model.to(device)
    for i in range(1):
          train_loss = 0
          for batch_idx, (data, _) in enumerate(train_loader):
          #   if(batch_idx>10):
            #      break
              data = data.to(device)
              optimizer.zero_grad()
              recon_batch, mu, logvar = model(data)
              loss = loss_function(recon_batch, data, mu, logvar)
              loss.backward()
              train_loss += loss.item()
              optimizer.step()
              del data
       # if batch_idx % args.log_interval == 0:
       #     print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
       #         epoch, batch_idx * len(data), len(train_loader.dataset),
       #         100. * batch_idx / len(train_loader),
       #         loss.item() / len(data)))

    #print('====> Epoch: {} Average loss: {:.4f}'.format(
  #        epoch, train_loss / len(train_loader.dataset)))


def test(model,test_loader):
    model.eval()
    test_loss = 0
    device = torch.device("cuda" if   torch.cuda.is_available() else "cpu")
    model.to(device)
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
          #  if(i>10):
           #     break
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()
            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n],
                                      recon_batch.view(batchSize, 3, 32, 32)[:n]])
                save_image(comparison.cpu(),
                             path+ 'reconstruction_' + str(os.getpid()) + '.png', nrow=n)
            del data
    test_loss /= len(test_loader.dataset)
    gc.collect()
  #  tune.report(loss=test_loss)
 #   print('====> Test set loss: {:.4f}'.format(test_loss))
    return test_loss

#   if __name__ == "__main__":
#      for epoch in range(1, args.epochs + 1):
  #      train(epoch)
 #       test(epoch)
#        with torch.no_grad():
#             sample = torch.randn(64,int(config.get("d3", 0))).to(device)
#              sample = model.decode(sample).cpu()
#               save_image(sample.view(64, 3, 96, 96),'/home/antoine/Projet/NovelTuning/sample_' + str(epoch) + '.png')







algo = HyperOptSearch(metric="loss",
    mode="min")
name="hypher"

#algo = ConcurrencyLimiter(algo, max_concurrent=1)
#os.makedirs(os.path.dirname(MODEL_PATH), exist_ok=True)
scheduler = PB2(
    time_attr="training_iteration",
    perturbation_interval=3,
    hyperparam_bounds={
        # distribution for resampling
     "lr": [1, 8] #tune.uniform(1e-4, 0.1 ),#,1e-4), #*10
,     "wd":[1, 10]#tune.uniform(1, 5)#,1e-4), #*10 et 0
,     "b1": [0, 2]#,1e-4), #*10 et 0
 ,    "b2": [1, 5] #,1e-4), #*10 et 0
 ,    "eps": [2, 10]#,1e-4), #*10 et 0
    }) 

#   hyperparam_bounds={
#        # distribution for resampling
#     "lr":  lambda: np.random.uniform(1, 8) #tune.uniform(1e-4, 0.1 ),#,1e-4), #*10
#,     "wd":lambda: np.random.uniform(1, 10)#tune.uniform(1, 5)#,1e-4), #*10 et 0
#,     "b1": lambda: np.random.uniform(0, 2)#,1e-4), #*10 et 0
# ,    "b2": lambda: np.random.uniform(1, 5)#,1e-4), #*10 et 0
# ,    "eps": lambda: np.random.uniform(2, 10)#,1e-4), #*10 et 0
#    })

imageSize = 32

cst=imageSize
total=cst*cst*3
def trial_name_id(trial):
    return f"{trial.trainable_name}_{trial.trial_id}"
from ray import tune

analysis = tune.run(
PytorchTrainable,
name=name,
scheduler=scheduler,
reuse_actors=True,
search_alg=algo,
verbose=2,
checkpoint_at_end=True,
num_samples=1024,
# export_formats=[ExportFormat.MODEL],
            config= {
 "lr":  tune.uniform(2.5, 8) #tune.uniform(1e-4, 0.1 ),#,1e-4), #*10
,     "wd":tune.uniform(1, 10)#tune.uniform(1, 5)#,1e-4), #*10 et 0
,     "b1": tune.uniform(0, 2)#,1e-4), #*10 et 0
 ,    "b2": tune.uniform(1, 5)#,1e-4), #*10 et 0
 ,    "eps": tune.uniform(2, 10)#,1e-4), #*10 et 0
# ,    "d1": tune.uniform(27648/6, 27648/3)
# ,    "d2": tune.uniform(27648/12, 27648/6)
# ,    "d3": tune.uniform(27648/24, 27648/12)
# ,    "d4": tune.uniform(27648/12, 27648/6)
#,    "d5": tune.uniform(27648/6, 27648/3)

                
    },      stop={
        "training_iteration": 10,
    },        metric="loss",
    mode="min",resources_per_trial={'gpu': 1}
              loggers=[TestLogger])
)
#all_trials = analysis.trials
#checkpoint_paths = [
#    os.path.join(analysis.get_best_checkpoint(t), "checkpoint")
#    for t in all_trials
#] 
#demo_gan(checkpoint_paths) in
